# SWEEM Model Implementation

This file is used to illustrate the preprocessing, training, and evaluation 
stages of our model. Comments and more information will be provided per section.

## Preprocessing

Here we load in the data and establish our train-test split. We also set up dataloaders
for us to be able to properly use the data within our training loop.

In [156]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset

data = pd.read_csv('./Data/OmicsData/data.csv')
data = pd.concat([data.iloc[:, :5541], data.iloc[:, -2:]], axis=1)

# Split the data into train and validation sets.
# Train test validaiton split is 80 10 10
train_data, test_data, train_labels, test_labels = train_test_split(
    data.iloc[:, 1:-2], data.iloc[:, -2:], test_size=0.2, random_state=42)

test_data, validation_data, test_labels, validation_labels = train_test_split(
    test_data, test_labels, test_size=0.5, random_state=42)

print(data.shape)
print(train_data.shape)
print(test_data.shape)
print(validation_data.shape)

# Number of genes in rna: 5540
# Number of genes in scna: 5507
# Number of genes in methy: 4846
# Total number of samples in the final dataset: 475

#rna
print(train_data.columns[0])
print(train_data.columns[5539])
# #scna
# print(train_data.columns[5540])
# print(train_data.columns[11046])
# #methy
# print(train_data.columns[11047])
# print(train_data.columns[15892])

#labels
print(train_labels.columns)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(475, 5543)
(380, 5540)
(47, 5540)
(48, 5540)
ZYX_rna
A2M_rna
Index(['OS_DAYS', 'OS_EVENT'], dtype='object')


In [157]:
# Create Tensor datasets
train_dataset = TensorDataset(torch.tensor(train_data.values), torch.tensor(train_labels.values))
val_dataset   = TensorDataset(torch.tensor(validation_data.values), torch.tensor(validation_labels.values))
test_dataset  = TensorDataset(torch.tensor(test_data.values), torch.tensor(test_labels.values))

# Create DataLoader objects
batch_size = 380
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
test_dataloader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

## Define Self-Attention Model

In [159]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

class LinearRegression(nn.Module):
    def __init__(self, input_size):
        super(LinearRegression, self).__init__()
        self.linear1 = nn.Linear(input_size, 256)
        self.linear2 = nn.Linear(256, 1)
        self.linearOut = nn.Linear(2, 1)
        self.relu = nn.ReLU()
        self.identity = nn.Identity()

    def forward(self, x, x2):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.relu(out)
        out = torch.cat((out, x2), 1)
        out = self.linearOut(out)
        # out = torch.sigmoid(out)
        return out

## Train Loop with Alt Dataloader

In [172]:
### Training Loop
from model import SelfAttentionModel
from loss import temp_loss, neg_par_log_likelihood
num_epochs = 50
epoch_train_losses = []
epoch_val_losses   = []

# model = LinearRegression(5540)
model = SelfAttentionModel(5540, 1024, 32)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Running on", device)
model.to(device)

criterion = neg_par_log_likelihood
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(num_epochs):
    epoch_train_loss = 0
    epoch_val_loss   = 0
    print(f"Epoch {epoch + 1} training:")
    progress_bar = tqdm(range(len(train_dataloader)))

    ## Training
    model.train()
    for (batchX, batchY) in train_dataloader:   
        batchX = batchX.to(torch.float32).to(device)
        time = batchY[:,0].to(torch.float32).to(device).reshape(-1, 1)
        event = batchY[:,1].to(torch.float32).to(device).reshape(-1, 1)
        # print("batchX", batchX[:, :5])
        # print("time", time)
        # print("event", event)
        
        outputs = model(batchX, event)
        # print("predictions", outputs)
        
        # print(outputs.shape, time.shape, event.shape)
        loss = criterion(outputs, time, event)
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        epoch_train_loss += loss.item()
        progress_bar.update(1)
    
    ## Validation
    model.eval()
    with torch.no_grad():
        for (batchX, batchY) in val_dataloader:
            batchX = batchX.to(torch.float32).to(device)
            time = batchY[:,0].to(torch.float32).to(device).reshape(-1, 1)
            event = batchY[:,1].to(torch.float32).to(device).reshape(-1, 1)
            outputs = model(batchX, event)
            loss = criterion(outputs, time, event)
            epoch_val_loss += loss.item()

    # Save and print losses
    epoch_train_loss /= len(train_dataloader)
    epoch_val_loss /= len(val_dataloader)
    epoch_train_losses.append(epoch_train_loss)
    epoch_val_losses.append(epoch_val_loss)
    print(f"Epoch {epoch + 1} training loss: {epoch_train_loss}")
    print(f"Epoch {epoch + 1} validation loss: {epoch_val_loss}")

Running on cpu
Epoch 1 training:


100%|██████████| 1/1 [00:06<00:00,  6.67s/it]


Epoch 1 training loss: 4.920126438140869
Epoch 1 validation loss: 3.359114646911621
Epoch 2 training:


100%|██████████| 1/1 [00:00<00:00, 29.20it/s]


Epoch 2 training loss: 4.933938503265381
Epoch 2 validation loss: 3.3591151237487793
Epoch 3 training:


...

Epoch 48 training loss: 5.0894455909729
Epoch 48 validation loss: 3.3591151237487793
Epoch 49 training:


100%|██████████| 1/1 [00:00<00:00, 36.95it/s]


Epoch 49 training loss: 4.9651875495910645
Epoch 49 validation loss: 3.3591151237487793
Epoch 50 training:


100%|██████████| 1/1 [00:00<00:00, 33.33it/s]


Epoch 50 training loss: 5.0098795890808105
Epoch 50 validation loss: 3.3591151237487793


In [ ]:
### Sanity Checking Outputs

# Check the outputs of the model on the test set
model.eval()
sample_number = 5
for test_images, test_labels in test_dataloader: 
    outputs = model(test_images.to(device).to(torch.float32))
    print("Intended Output: ", test_labels)
    print("Actual Output: ", outputs)
    break